Created by Pooja Subramaniam and Marc Aurel Vischer on Tue, May 8.

In [1]:
import urllib3
from bs4 import BeautifulSoup

In [2]:
#These are the urls referring directly to high, low temperature
hi_lo_url = "https://wetter.bild.de/web2014/ifr-wetter-deutschland.asp"
prec_url = "https://wetter.bild.de/web2014/ifr-niederschlag-deutschland.asp"

In [3]:
#load and parse page
http = urllib3.PoolManager()
hi_lo_bs = BeautifulSoup(http.request('GET', hi_lo_url).data, "lxml")
prec_bs = BeautifulSoup(http.request('GET',prec_url).data, "lxml")

#print(hi_lo.prettify())

/home/bccn2017/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/bccn2017/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [4]:
#TEMPERATURE HIGH/LOW, bild has today + 5 days forecast for that
#iterate over days, extract day layer for each
temp_dicts = []
for day in range(5):
    # extract current day layer
    day_layer = hi_lo_bs.find_all('div', id="wk_layer_wr{}".format(day))
    #print(day_layer[0]['id'])
    if len(day_layer)!=1:
        raise Exception("Found more than one layer for single day.")
        
    # extract all the cities from that layer
    day_cities = day_layer[0].find_all('div', class_="wk_map_text")
    day_dict = {}
    for city in day_cities:
        hi_lo_str = city.nobr.nextSibling.nextSibling
        high = int(hi_lo_str.split('|')[0].split('°')[0])
        low = int(hi_lo_str.split('|')[1].split('°')[0])
        day_dict[city.nobr.string] = (high, low)
    temp_dicts.append(day_dict)

In [5]:
#PRECIPITATION,  bild has only today + 3 days forecast for that
#iterate over days, extract day layer for each
prec_dicts = []
for day in range(3):
    # extract current day layer
    day_layer = prec_bs.find_all('div', id="wk_layer_wr{}".format(day))
    #print(day_layer[0]['id'])
    if len(day_layer)!=1:
        raise Exception("Found more than one layer for single day.")
        
    # extract all the cities from that layer
    day_cities = day_layer[0].find_all('div', class_="wk_map_text")
    day_dict = {}
    for city in day_cities:
        prec_str = city.nobr.nextSibling.nextSibling
        prec_value = int(prec_str[:-2])
        day_dict[city.nobr.string] = prec_value
    prec_dicts.append(day_dict)